In [ ]:
import os
import torch

os.chdir("/n/home08/zkong/mufan/tmp/moebench/open-instruct")
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

logits = torch.load("output/logits/baseline.pt")
print("new logits shape", logits.shape)

/tmp/ipykernel_2579344/2603555335.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  logits = torch.load("output/logits/baseline.pt")


new logits shape torch.Size([16, 2693448, 64])
selected_experts shape torch.Size([16, 2693448, 8])


In [14]:
selected_experts = torch.topk(logits, k=8, dim=-1).indices
print("selected_experts shape", selected_experts.shape)
expert_frequency = torch.zeros((selected_experts.shape[0], 64), dtype=torch.int32)
for i in range(selected_experts.shape[0]):
    counts = torch.bincount(selected_experts[i].flatten(), minlength=64)
    expert_frequency[i] = counts
print("expert_frequency shape", expert_frequency.shape)
print("expert_frequency", expert_frequency)

selected_experts shape torch.Size([16, 2693448, 8])
expert_frequency shape torch.Size([16, 64])
expert_frequency tensor([[837906, 154755, 173987,  ..., 348036, 173925, 492011],
        [274722, 665449, 211892,  ..., 220395, 358158, 429618],
        [246277, 669706,  89201,  ..., 220317, 326267, 408857],
        ...,
        [279226, 417342, 299125,  ..., 459584,  65043, 370525],
        [268925, 416823, 506604,  ..., 156116, 415473, 488598],
        [135508, 189362, 441918,  ..., 424252, 297283, 569011]],
       dtype=torch.int32)


In [16]:
import torch


def calculate_entropy(A):
    A = A.float()
    row_sums = A.sum(dim=-1, keepdim=True)
    P = A / (row_sums + 1e-10)
    P_log_P = P * torch.log2(P + 1e-10)
    entropy = -P_log_P.sum(dim=-1)
    return entropy


print("entropy", calculate_entropy(expert_frequency))
print("average entropy", calculate_entropy(expert_frequency).mean())
print("max entropy", calculate_entropy(expert_frequency).max())

entropy tensor([5.7442, 5.8317, 5.8564, 5.8119, 5.8006, 5.7841, 5.8014, 5.8049, 5.8122,
        5.8377, 5.8217, 5.8190, 5.8210, 5.8035, 5.8613, 5.8114])
average entropy tensor(5.8139)
max entropy tensor(5.8613)


In [12]:
from tqdm.auto import tqdm

top_expert_frequency = []
for layer in tqdm(range(logits.shape[0]), desc="layer"):
    top1_experts = torch.argmax(logits[layer], dim=1)  # 形状为 [2693448]

    expert_logits = []
    for expert_idx in tqdm(range(logits.shape[2]), desc="expert", disable=True):
        # 创建布尔掩码，标记出当前专家为 Top-1 的位置
        mask = top1_experts == expert_idx  # 形状为 [2,693,448]
        # 使用掩码筛选出对应的 logits
        selected_logits = logits[layer, mask]
        expert_logits.append(selected_logits)
        # print("selected_logits shape", selected_logits.shape)
    top_expert_frequency.append(expert_logits)

layer:   0%|          | 0/16 [00:00<?, ?it/s]

In [19]:
collaboration = torch.zeros(
    (logits.shape[0], logits.shape[2], logits.shape[2] - 1), dtype=torch.int32
)
for layer in tqdm(range(logits.shape[0]), desc="layer"):
    for expert in tqdm(range(logits.shape[2]), desc="expert", disable=True):
        expert_logits = top_expert_frequency[layer][expert]
        expert_logits[:, expert] = float("-inf")
        top_experts = torch.topk(expert_logits, k=7, dim=-1).indices
        collaboration_frequency = torch.bincount(top_experts.flatten(), minlength=64)
        mask = torch.arange(64) != expert
        collaboration[layer, expert] = collaboration_frequency[mask]
print(collaboration)
print("entropy", calculate_entropy(collaboration))
print("average entropy", calculate_entropy(collaboration).mean())
print("shape of entropy", calculate_entropy(collaboration).shape)

layer:   0%|          | 0/16 [00:00<?, ?it/s]

tensor([[[  436,  1108,  1298,  ...,  3279,  1193,   574],
         [ 4937,  4048,  4923,  ...,  5503,  2687,  2583],
         [14547,  1325,  3942,  ...,  1808,  1982,  2361],
         ...,
         [ 4700,  5978,  6320,  ...,  1627,  2528,  2085],
         [ 5737,  1813,  1958,  ...,  1149,  7014,  5733],
         [   71,   190,    66,  ...,  4196,   794,   229]],

        [[31251,  6176, 25945,  ...,  8402,  6312, 12937],
         [11837, 10973, 15579,  ...,  1038,  3622,  7621],
         [ 1709,  5118,  1351,  ...,  1273,   404,   670],
         ...,
         [ 1925,   696,  1372,  ..., 14740, 18572, 17881],
         [  114,   969,   341,  ...,   668,   914,  2401],
         [  720,  3171,  1169,  ..., 26983,   670,  2270]],

        [[ 2614,  2189, 10781,  ...,  4094, 29594,  3557],
         [ 1072,   222,  2631,  ...,  1198,  1842,   225],
         [    4,     1,     8,  ...,   315,    38,    12],
         ...,
         [  959,    75,   268,  ...,  7891,  8631,   144],
         [